In [2]:
import logging

In [ ]:
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

: 

In [ ]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
print(os.getenv("GROQ_MODEL"))

llama-3.3-70b-versatile


: 

: 

In [ ]:

try:
    router_llm = ChatGroq(
        model=os.getenv("GROQ_MODEL"),
        api_key=os.getenv("GROQ_API_KEY")
    )
    logger.info("Successfully connected to the router LLM")
except Exception as e:
    logger.exception("Failed to connect to the router LLM")


2025-10-16 01:25:33,781 - __main__ - INFO - Successfully connected to the router LLM


: 

In [ ]:
answer = router_llm.invoke('Give me a financial tip but i want you to EXPLICITLY RETURN THE ANSWER IN A JSON FORMAT: {"tip": "The actual tip"}')

2025-10-16 01:27:30,542 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


: 

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
import logging

logger = logging.getLogger(__name__)

# Define your output schema
class FinancialTip(BaseModel):
    tip: str = Field(description="A practical financial tip")

# Initialize the LLM
try:
    router_llm = ChatGroq(
        model=os.getenv("GROQ_MODEL"),
        api_key=os.getenv("GROQ_API_KEY"),
        temperature=0.7
    )
    logger.info("✅ Successfully connected to Groq LLM")
except Exception as e:
    logger.exception("❌ Failed to connect to Groq LLM")
    raise

# Method 1: Using JsonOutputParser (RECOMMENDED)
def get_financial_tip_method1():
    """Best method: JsonOutputParser with structured schema"""
    
    parser = JsonOutputParser(pydantic_object=FinancialTip)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a financial advisor. {format_instructions}"),
        ("user", "{query}")
    ])
    
    chain = prompt | router_llm | parser
    
    try:
        result = chain.invoke({
            "query": "Give me a financial tip",
            "format_instructions": parser.get_format_instructions()
        })
        
        logger.info(f"✅ Received structured output: {result}")
        return result  # Returns: {"tip": "..."}
        
    except Exception as e:
        logger.exception("❌ Failed to get structured output")
        raise

# Method 2: Using with_structured_output (NEWER API)
def get_financial_tip_method2():
    """Alternative: Using Groq's structured output API"""
    
    structured_llm = router_llm.with_structured_output(FinancialTip)
    
    try:
        result = structured_llm.invoke("Give me a financial tip")
        
        # Result is already a Pydantic object
        logger.info(f"✅ Received Pydantic object: {result}")
        
        # Convert to dict if needed
        return result.dict()  # Returns: {"tip": "..."}
        
    except Exception as e:
        logger.exception("❌ Failed to get structured output")
        raise

# Method 3: Manual JSON parsing with retry logic (FALLBACK)
def get_financial_tip_method3():
    """Fallback: Manual JSON parsing with validation"""
    import json
    
    prompt = """You are a financial advisor. Provide a financial tip.

CRITICAL: Return ONLY valid JSON in this exact format:
{"tip": "your financial tip here"}

Do not include any other text, explanations, or markdown formatting."""
    
    max_retries = 3
    
    for attempt in range(max_retries):
        try:
            response = router_llm.invoke(prompt)
            content = response.content
            
            # Remove markdown code blocks if present
            content = content.strip()
            if content.startswith("```json"):
                content = content.replace("```json", "").replace("```", "").strip()
            elif content.startswith("```"):
                content = content.replace("```", "").strip()
            
            # Parse JSON
            result = json.loads(content)
            
            # Validate structure
            if "tip" not in result:
                raise ValueError("Missing 'tip' key in response")
            
            logger.info(f"✅ Successfully parsed JSON on attempt {attempt + 1}")
            return result
            
        except json.JSONDecodeError as e:
            logger.warning(f"⚠️ JSON parse error on attempt {attempt + 1}: {e}")
            logger.warning(f"Raw content: {content}")
            
            if attempt == max_retries - 1:
                logger.error("❌ All retry attempts exhausted")
                # Fallback: return plain text wrapped in JSON
                return {"tip": content}
        
        except Exception as e:
            logger.exception(f"❌ Unexpected error on attempt {attempt + 1}")
            raise

# Usage Examples
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    
    print("\n=== Method 1: JsonOutputParser ===")
    tip1 = get_financial_tip_method1()
    print(f"Tip: {tip1['tip']}")
    
    print("\n=== Method 2: with_structured_output ===")
    tip2 = get_financial_tip_method2()
    print(f"Tip: {tip2['tip']}")
    
    print("\n=== Method 3: Manual parsing with retry ===")
    tip3 = get_financial_tip_method3()
    print(f"Tip: {tip3['tip']}")

/home/theurieric/Desktop/DirectEd/main/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
2025-10-16 01:29:51,454 - __main__ - INFO - ✅ Successfully connected to Groq LLM



=== Method 1: JsonOutputParser ===


2025-10-16 01:29:52,266 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-16 01:29:52,280 - __main__ - INFO - ✅ Received structured output: {'tip': 'Start building an emergency fund to cover 3-6 months of living expenses to ensure financial stability and security.'}


Tip: Start building an emergency fund to cover 3-6 months of living expenses to ensure financial stability and security.

=== Method 2: with_structured_output ===


2025-10-16 01:29:52,610 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-16 01:29:52,613 - __main__ - INFO - ✅ Received Pydantic object: tip='Create a budget and track your expenses to understand where your money is going and make informed financial decisions.'


Tip: Create a budget and track your expenses to understand where your money is going and make informed financial decisions.

=== Method 3: Manual parsing with retry ===


2025-10-16 01:29:53,082 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-16 01:29:53,084 - __main__ - INFO - ✅ Successfully parsed JSON on attempt 1


Tip: Start building an emergency fund to cover at least 3-6 months of living expenses to ensure financial stability and avoid going into debt when unexpected expenses arise


: 

In [ ]:
class EventOutput(BaseModel):
    event_name: str = Field(..., description="Event name/title")
    location: str = Field(..., description="Event location")
    event_summary: str = Field(..., description="A brief description of the event..what it entails..who'll be there, price and the rest")
    fun_fact: str = Field(..., description="A fun fact about the event,theme or anything related to the event")


structured_llm = router_llm.with_structured_output(EventOutput)
structured_llm.invoke("Tell me an event happening in Nairobi this weekend").fun_fact

2025-10-16 01:34:45,234 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


'The festival will feature a surprise performance by a renowned African artist.'

: 

In [ ]:
structured_llm = router_llm.with_structured_output(FinancialTip)

: 

In [ ]:
import logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

: 

In [ ]:
from pathlib import Path
try:
    project_root = Path(__name__).resolve().parent.parent.parent
    pdf_path = f"{project_root}/knowledge"
    persist_directory = f"{project_root}/db"
    logger.info("Registered required directories")
except Exception as e:
    logger.exception("Failed to register required directories")

2025-10-17 19:20:05,132 - __main__ - INFO - Registered required directories


: 

In [ ]:
print(project_root)
config_dir = f"{project_root}/src/conturi/config"
print(config_dir)

/home/theurieric/Desktop/DirectEd/projects/conturi/backend
/home/theurieric/Desktop/DirectEd/projects/conturi/backend/src/conturi/config


: 

/home/theurieric/Desktop/DirectEd/projects/conturi/backend/src/conturi/config

In [ ]:
import requests
import json
PRIVATE_TOKEN = "d"
API_BASE_URL = "https://www.eventbriteapi.com/v3/"

: 

In [ ]:
import requests
import json

# --- Configuration ---
PRIVATE_TOKEN = "."
API_BASE_URL = "https://www.eventbriteapi.com/v3/"

# --- Set up headers for authentication ---
headers = {
    "Authorization": f"Bearer {PRIVATE_TOKEN}",
    "Content-Type": "application/json",
}

# --- Define search parameters ---
# You can search by keyword, location, date, etc.
# See docs for all options: https://www.eventbrite.com/platform/api#/reference/event/search
params = {
    "q": "tech",
    "location.address": "Nakuru, Kenya"
}

# --- Make the API request ---
response = requests.get(
    f"{API_BASE_URL}/events/search/",
    headers=headers,
    params=params
)

# --- Process the response ---
if response.status_code == 200:
    events_data = response.json()
    print("✅ Successfully found events.")
    print("-" * 30)
    
    # Check if any events were returned
    if not events_data.get("events"):
        print("No events found matching your criteria.")
    else:
        # Loop through the events and print details
        for event in events_data["events"]:
            event_name = event.get("name", {}).get("text", "N/A")
            start_time = event.get("start", {}).get("local", "N/A")
            is_free = event.get("is_free", False)
            url = event.get("url", "N/A")
            
            print(f"Event: {event_name}")
            print(f"  Starts: {start_time}")
            print(f"  Is Free: {'Yes' if is_free else 'No'}")
            print(f"  URL: {url}")
            print("-" * 30)

else:
    print(f"❌ Error fetching events: {response.status_code}")
    print(f"Error details: {response.text}")

❌ Error fetching events: 404
Error details: {"status_code":404,"error_description":"The path you requested does not exist.","error":"NOT_FOUND"}


: 